In [3]:
import polars as pl
import numpy as np

In [4]:
# Question 1
# 1a. Pivot data
economics_long = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/economics_long.csv")

economics_wide = economics_long.pivot(
    index="date",
    columns="variable",
    values="value"
).sort("date")

economics_wide

/tmp/ipython-input-1845098523.py:5: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  economics_wide = economics_long.pivot(


date,pce,pop,psavert,uempmed,unemploy
str,f64,f64,f64,f64,f64
"""1967-07-01""",506.7,198712.0,12.6,4.5,2944.0
"""1967-08-01""",509.8,198911.0,12.6,4.7,2945.0
"""1967-09-01""",515.6,199113.0,11.9,4.6,2958.0
"""1967-10-01""",512.2,199311.0,12.9,4.9,3143.0
"""1967-11-01""",517.4,199498.0,12.8,4.7,3066.0
…,…,…,…,…,…
"""2014-12-01""",12062.0,319746.157,7.6,12.9,8717.0
"""2015-01-01""",12046.0,319928.646,7.7,13.2,8903.0
"""2015-02-01""",12082.4,320074.511,7.9,12.9,8610.0


In [5]:
#1b. Calculate median unemployment for 2010-01-01 and beyond
median_unemploy = (
    economics_wide
    .filter(pl.col("date") >= "2010-01-01")
    .select(pl.col("unemploy").median().alias("median_unemploy"))
).item()

print(f"The median unemployment for 2010-01-01 and beyond is: {median_unemploy}")


The median unemployment for 2010-01-01 and beyond is: 12471.0


In [6]:
# Question 2: Air Passenger data
# 2a. Unpivot data
air_passengers = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/AirPassengers.txt",
    separator="\t"
)

month_cols = [c for c in air_passengers.columns if c.lower() != "year"]

air_passengers_long = (
    air_passengers
    .unpivot(
        on=month_cols,
        index=["Year"],
        variable_name="month",
        value_name="passengers"
    )
    .rename({"Year": "year"})
    .with_columns([
        pl.col("year").cast(pl.Int32),
        pl.col("month").str.to_lowercase(),
        pl.col("passengers").cast(pl.Int64)
    ])
    .sort(["year","month"])
)

air_passengers_long

year,month,passengers
i32,str,i64
1949,"""apr""",129
1949,"""aug""",148
1949,"""dec""",118
1949,"""feb""",118
1949,"""jan""",112
…,…,…
1960,"""mar""",419
1960,"""may""",472
1960,"""nov""",390


In [7]:
# 2b. Find the standard deviation between 1955 and 1960
std_passengers = (
    air_passengers_long
    .filter((pl.col("year") >= 1955) & (pl.col("year") <= 1960))
    .select(pl.col("passengers").std().alias("std_passengers"))
).item()

print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_passengers:.2f}")

The standard deviation of airline passengers between 1955 and 1960 is: 86.44


In [8]:
# Question 3: WHO tuberculosis case data
#3a. Pivot data
table2 = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/table2.csv")

table2_wide = (
    table2
    .pivot(
        index=["country","year"],
        columns="type",
        values="count"
    )
    .sort(["country","year"])
)

table2_wide

/tmp/ipython-input-3894968542.py:7: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


country,year,cases,population
str,i64,i64,i64
"""Afghanistan""",1999,745,19987071
"""Afghanistan""",2000,2666,20595360
"""Brazil""",1999,37737,172006362
"""Brazil""",2000,80488,174504898
"""China""",1999,212258,1272915272
"""China""",2000,213766,1280428583


In [9]:
#3b. Calculate highest cases/population ratio for 1999
table2_with_ratio = table2_wide.with_columns(
    (pl.col("cases") / pl.col("population")).alias("ratio")
)

year_1999 = table2_with_ratio.filter(pl.col("year") == 1999).sort("ratio", descending=True)

highest_ratio_country = year_1999["country"][0]

print("Data for 1999 with ratios:")
print(year_1999)
print(f"\nThe country with the highest cases/population ratio in 1999 is: {highest_ratio_country}")

Data for 1999 with ratios:
shape: (3, 5)
┌─────────────┬──────┬────────┬────────────┬──────────┐
│ country     ┆ year ┆ cases  ┆ population ┆ ratio    │
│ ---         ┆ ---  ┆ ---    ┆ ---        ┆ ---      │
│ str         ┆ i64  ┆ i64    ┆ i64        ┆ f64      │
╞═════════════╪══════╪════════╪════════════╪══════════╡
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  ┆ 0.000219 │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 ┆ 0.000167 │
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   ┆ 0.000037 │
└─────────────┴──────┴────────┴────────────┴──────────┘

The country with the highest cases/population ratio in 1999 is: Brazil
